# Посещение врачей

In [15]:
import pandas as pd
import numpy as np

# игнорирование предупреждения о формате при импорте из excel
import warnings

# warnings.simplefilter("ignore")

%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


ISO-8859-1

# Если нестандартные кодировки:

In [19]:
# загрузка файла для ЕАО !!!
df_data = pd.read_csv(
    "/home/sk27/work/med/data/source.csv",
    sep=";",
    encoding="utf-16",
    low_memory=False,
)

In [20]:
# загрузка справочников
f_032 = pd.read_excel("/home/sk27/work/med/data/F032.xlsx")
v_021 = pd.read_excel("/home/sk27/work/med/data/V021.xlsx")
v_002 = pd.read_excel("/home/sk27/work/med/data/V002.xlsx")
v_006 = pd.read_excel("/home/sk27/work/med/data/V006.xlsx")
v_025 = pd.read_excel("/home/sk27/work/med/data/V025.xlsx")
oid_data = pd.read_excel("/home/sk27/work/med/data/oid_data.xlsx")

# актуализация справочников
f_032 = f_032[["OKTMO_P", "SUBJ"]].drop_duplicates().reset_index(drop=True)
v_021 = v_021[v_021["DATEEND"].isna()]
v_002 = v_002[v_002["DATEEND"].isna()]
v_006 = v_006[v_006["DATEEND"].isna()]
v_025 = v_025[v_025["DATEEND"].isna()]

## Заголовок добавить вручную
### Переходить сразу к расшифровке кодов

In [21]:
df_data

,year,month,region,oktmo,clinic,oid,depart,branch,doctor_specialty,doctor_profile,...,purpose,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum
0,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,1.0,B01.014.001,137-097-107 64,24,790001,79000100,1,1,1,NaN
1,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,1.3,R03.03.102.001,137-097-107 64,24,790001,79000100,1,1,1,NaN
2,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,1.0,B01.014.001,137-097-107 64,24,790001,79000100,10,1,1,NaN
3,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,3.0,B01.014.002,137-097-107 64,24,790001,79000100,12,5,1,NaN
4,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,3.0,B01.014.002,138-993-968 45,24,790001,79000100,2,15,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25405,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,NaN,41,60,...,1.0,B01.027.001,17717264189,41,790021,79002100,2,3,1,NaN
25406,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,NaN,45,162,...,1.0,B01.028.001,13109077827,45,790021,79002100,1,1,1,NaN
25407,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,NaN,76,97,...,1.0,B01.047.001,6180038532,76,790021,79002100,1,1,1,NaN
25408,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,NaN,76,97,...,1.0,B01.047.001,6180038532,76,790021,79002100,1,1,1,NaN


In [22]:
df_data.shape

(25410, 23)

In [23]:
df_data[df_data.duplicated()]

,year,month,region,oktmo,clinic,oid,depart,branch,doctor_specialty,doctor_profile,...,purpose,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum


In [24]:
df_data[df_data.duplicated()].shape

(0, 23)

# Обычные кодировки
## 1. Подготовка данных

In [46]:
%%time

# вспомогательные функции

def oktmo(o):
    """
    функция дополняет код ОКТМО до 11 символов или до 10 символов без ведущего 0
    """
    if len(str(o)) == 1:
        l = 9
    else:
        l = 11-len(str(o))
    return int(str(o) + "0"*(l))
    
def visit_count(row):
    """
    функция определяет количество посещений согласно условиям
    """
    if row["purpose"] >= 3:
        res = row["code_usl"] * row["kol_usl"]
    else:
        res = row["sl_id"]
    if np.isnan(res):
        return "не определено"
    else:
        return res
    
# загрузка справочников
f_032 = pd.read_excel("/home/sk27/work/med/data/F032.xlsx")
v_021 = pd.read_excel("/home/sk27/work/med/data/V021.xlsx")
v_002 = pd.read_excel("/home/sk27/work/med/data/V002.xlsx")
v_006 = pd.read_excel("/home/sk27/work/med/data/V006.xlsx")
v_025 = pd.read_excel("/home/sk27/work/med/data/V025.xlsx")
oid_data = pd.read_excel("/home/sk27/work/med/data/oid_data.xlsx")

# актуализация справочников
f_032 = f_032[["OKTMO_P", "SUBJ"]].drop_duplicates().reset_index(drop=True)
v_021 = v_021[v_021["DATEEND"].isna()]
v_002 = v_002[v_002["DATEEND"].isna()]
v_006 = v_006[v_006["DATEEND"].isna()]
v_025 = v_025[v_025["DATEEND"].isna()]


# добавление в файл данных строки заголовков
with open("/home/sk27/work/med/data/source.csv", "r") as f:
    first_line = next(f).strip()
    if first_line[0:4] != "year":
        with open(
            "/home/sk27/work/med/data/source.csv", "r", encoding="ISO-8859-1"
        ) as original:
            data = original.read()
        with open("/home/sk27/work/med/data/source.csv", "w") as modified:
            modified.write(
                "year;month;region;clinic;oid;depart;branch;doctor_specialty;doctor_profile;terms;doctor_id;diagnosis;purpose;service_id;usl_service_id;usl_doctor_specialty;usl_clinic;usl_depart;sl_id;code_usl;kol_usl;yet_sum\n"
                + data
            )
    else:
        pass


# загрузка файла
df_data = pd.read_csv(
    "/home/sk27/work/med/data/source.csv",
    sep=";",
    encoding="ISO-8859-1",
    low_memory=False,
)

# УДАЛЕНИЕ ДУБЛИКАТОВ СТРОК
df_data = df_data.drop_duplicates()

# удаление кавычек
for i, col in enumerate(df_data.columns):
    df_data.iloc[:, i] = df_data.iloc[:, i].astype("str").str.replace('"', "")
# df_data = df_data.apply(pd.to_numeric, errors="ignore")

# замена ВСЕХ запятых точками
for i, col in enumerate(df_data.columns):
    df_data.iloc[:, i] = df_data.iloc[:, i].astype("str").str.replace(',', ".")
df_data = df_data.apply(pd.to_numeric, errors="ignore")

# вставка столбца ОКТМО
df_data.insert(3, "oktmo", df_data["region"].apply(oktmo))

# проверка загруженных данных

# а) проверка количества столбцов - их должно быть именно 23 вместе с добавленным мной столбцом ОКТМО
if df_data.shape[1] > 23:
    print(f"Количество полей в файле (их {df_data.shape[1]}) не соответствует форме отчёта, должно быть 23.")
else:
    pass

# б) проверка столбца terms на "только 3"
try:
    if df_data["terms"].mean() != 3:
        print(
            "Нарушено требование об указании в поле 'Условия оказания медпомощи' только значения '3'."
        )
except:
    print("Не проходит проверка на '3'")
    
# расшифровка кодов

CPU times: user 50.6 s, sys: 224 ms, total: 50.8 s
Wall time: 50.9 s


In [87]:
df_data.head(10)

,year,month,region,oktmo,clinic,oid,depart,branch,doctor_specialty,doctor_profile,...,purpose,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum
0,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,1.0,B01.014.001,137-097-107 64,24,790001,79000100,1,1,1,NaN
1,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,1.3,R03.03.102.001,137-097-107 64,24,790001,79000100,1,1,1,NaN
2,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,1.0,B01.014.001,137-097-107 64,24,790001,79000100,10,1,1,NaN
3,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,3.0,B01.014.002,137-097-107 64,24,790001,79000100,12,5,1,NaN
4,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,3.0,B01.014.002,138-993-968 45,24,790001,79000100,2,15,1,NaN
5,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,1.0,B01.014.001,137-097-107 64,24,790001,79000100,1,1,1,NaN
6,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,1.0,B01.014.001,137-097-107 64,24,790001,79000100,1,1,1,NaN
7,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,3.0,B01.014.001,137-097-107 64,24,790001,79000100,1,4,1,NaN
8,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,3.0,B01.014.002,137-097-107 64,24,790001,79000100,7,3,1,NaN
9,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,3.0,B01.014.002,137-097-107 64,24,790001,79000100,3,3,1,NaN


In [88]:
df_data.shape

(25410, 23)

In [89]:
df_data[df_data.duplicated()]

,year,month,region,oktmo,clinic,oid,depart,branch,doctor_specialty,doctor_profile,...,purpose,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum


## 2. Расшифровка кодов

In [90]:
# расшифорвка названия субьекта Росссийской Федерации
df_data = df_data.merge(f_032, left_on="oktmo", right_on="OKTMO_P", how="left")

In [91]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "depart",
        "branch",
        "doctor_specialty",
        "doctor_profile",
        "terms",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
    ]
].reset_index(drop=True)

In [92]:
df_data = df_data.drop_duplicates()

In [93]:
df_data.shape

(25410, 24)

In [94]:
# расшифровка наименования (сокращённого) медучреждения
df_data = df_data.merge(oid_data, left_on="oid", right_on="oid", how="left")

In [95]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "nameShort",
        "depart",
        "branch",
        "doctor_specialty",
        "doctor_profile",
        "terms",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
    ]
].reset_index(drop=True)

In [96]:
df_data.shape

(25410, 25)

In [97]:
# расшифровка специальности врача
df_data = df_data.merge(
    v_021, left_on="doctor_specialty", right_on="IDSPEC", how="left"
)

In [98]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "nameShort",
        "depart",
        "branch",
        "doctor_specialty",
        "POSTNAME",
        "doctor_profile",
        "terms",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
    ]
].reset_index(drop=True)

In [99]:
df_data.shape

(25410, 26)

In [100]:
# расшифровка профиля врача
df_data = df_data.merge(v_002, left_on="doctor_profile", right_on="IDPR", how="left")

In [101]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "nameShort",
        "depart",
        "branch",
        "doctor_specialty",
        "POSTNAME",
        "doctor_profile",
        "PRNAME",
        "terms",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
    ]
].reset_index(drop=True)

In [102]:
df_data.shape

(25410, 27)

In [103]:
# расшифровка специальности врача, оказавшего услугу
df_data = df_data.merge(
    v_021, left_on="doctor_specialty", right_on="IDSPEC", how="left"
)

In [104]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "nameShort",
        "depart",
        "branch",
        "doctor_specialty",
        "POSTNAME_x",
        "doctor_profile",
        "PRNAME",
        "terms",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "POSTNAME_y",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
    ]
].reset_index(drop=True)

In [105]:
df_data.shape

(25410, 28)

In [106]:
# расшифровка условий оказания помощи
df_data = df_data.merge(v_006, left_on="terms", right_on="IDUMP", how="left")

In [107]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "nameShort",
        "depart",
        "branch",
        "doctor_specialty",
        "POSTNAME_x",
        "doctor_profile",
        "PRNAME",
        "terms",
        "UMPNAME",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "POSTNAME_y",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
    ]
].reset_index(drop=True)

In [108]:
df_data.shape

(25410, 29)

In [109]:
# расшифровка цели посещения
df_data = df_data.merge(v_025, left_on="purpose", right_on="IDPC", how="left")

In [110]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "nameShort",
        "depart",
        "branch",
        "doctor_specialty",
        "POSTNAME_x",
        "doctor_profile",
        "PRNAME",
        "terms",
        "UMPNAME",
        "doctor_id",
        "diagnosis",
        "purpose",
        "N_PC",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "POSTNAME_y",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
    ]
].reset_index(drop=True)

In [111]:
df_data.shape

(25410, 30)

In [112]:
df_data

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,service_id,usl_service_id,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum
0,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,NaN,...,B01.014.001,137-097-107 64,24,врач-инфекционист,790001,79000100,1,1,1,NaN
1,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,NaN,...,R03.03.102.001,137-097-107 64,24,врач-инфекционист,790001,79000100,1,1,1,NaN
2,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,NaN,...,B01.014.001,137-097-107 64,24,врач-инфекционист,790001,79000100,10,1,1,NaN
3,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,NaN,...,B01.014.002,137-097-107 64,24,врач-инфекционист,790001,79000100,12,5,1,NaN
4,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,NaN,...,B01.014.002,138-993-968 45,24,врач-инфекционист,790001,79000100,2,15,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25405,2023,2,99,99000000000,Еврейская АО,790021,1.2.643.5.1.13.13.12.2.79.8838,"ОГБУЗ ""Онкологический диспансер""",79002100,NaN,...,B01.027.001,17717264189,41,врач-онколог,790021,79002100,2,3,1,NaN
25406,2023,2,99,99000000000,Еврейская АО,790021,1.2.643.5.1.13.13.12.2.79.8838,"ОГБУЗ ""Онкологический диспансер""",79002100,NaN,...,B01.028.001,13109077827,45,врач-оториноларинголог,790021,79002100,1,1,1,NaN
25407,2023,2,99,99000000000,Еврейская АО,790021,1.2.643.5.1.13.13.12.2.79.8838,"ОГБУЗ ""Онкологический диспансер""",79002100,NaN,...,B01.047.001,6180038532,76,врач-терапевт,790021,79002100,1,1,1,NaN
25408,2023,2,99,99000000000,Еврейская АО,790021,1.2.643.5.1.13.13.12.2.79.8838,"ОГБУЗ ""Онкологический диспансер""",79002100,NaN,...,B01.047.001,6180038532,76,врач-терапевт,790021,79002100,1,1,1,NaN


## 3. Расчёт количества посещений

In [113]:
# определение количества посещений
df_data["visit_count"] = df_data.apply(visit_count, axis=1)

In [114]:
df_data

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_service_id,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,visit_count
0,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,NaN,...,137-097-107 64,24,врач-инфекционист,790001,79000100,1,1,1,NaN,1
1,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,NaN,...,137-097-107 64,24,врач-инфекционист,790001,79000100,1,1,1,NaN,1
2,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,NaN,...,137-097-107 64,24,врач-инфекционист,790001,79000100,10,1,1,NaN,10
3,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,NaN,...,137-097-107 64,24,врач-инфекционист,790001,79000100,12,5,1,NaN,5
4,2023,1,99,99000000000,Еврейская АО,790001,1.2.643.5.1.13.13.12.2.79.8850,ОГБУЗ Областная больница,79000100,NaN,...,138-993-968 45,24,врач-инфекционист,790001,79000100,2,15,1,NaN,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25405,2023,2,99,99000000000,Еврейская АО,790021,1.2.643.5.1.13.13.12.2.79.8838,"ОГБУЗ ""Онкологический диспансер""",79002100,NaN,...,17717264189,41,врач-онколог,790021,79002100,2,3,1,NaN,2
25406,2023,2,99,99000000000,Еврейская АО,790021,1.2.643.5.1.13.13.12.2.79.8838,"ОГБУЗ ""Онкологический диспансер""",79002100,NaN,...,13109077827,45,врач-оториноларинголог,790021,79002100,1,1,1,NaN,1
25407,2023,2,99,99000000000,Еврейская АО,790021,1.2.643.5.1.13.13.12.2.79.8838,"ОГБУЗ ""Онкологический диспансер""",79002100,NaN,...,6180038532,76,врач-терапевт,790021,79002100,1,1,1,NaN,1
25408,2023,2,99,99000000000,Еврейская АО,790021,1.2.643.5.1.13.13.12.2.79.8838,"ОГБУЗ ""Онкологический диспансер""",79002100,NaN,...,6180038532,76,врач-терапевт,790021,79002100,1,1,1,NaN,1


## 4. Контроль результатов

In [115]:
# формат данных
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25410 entries, 0 to 25409
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  25410 non-null  int64  
 1   month                 25410 non-null  int64  
 2   region                25410 non-null  int64  
 3   oktmo                 25410 non-null  int64  
 4   SUBJ                  25410 non-null  object 
 5   clinic                25410 non-null  int64  
 6   oid                   25410 non-null  object 
 7   nameShort             25410 non-null  object 
 8   depart                25410 non-null  int64  
 9   branch                377 non-null    float64
 10  doctor_specialty      25410 non-null  int64  
 11  POSTNAME_x            25410 non-null  object 
 12  doctor_profile        25410 non-null  int64  
 13  PRNAME                25410 non-null  object 
 14  terms                 25410 non-null  int64  
 15  UMPNAME            

In [116]:
# пустые количества посещений
df_data[np.isnan(df_data["visit_count"])]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_service_id,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,visit_count


In [117]:
# не определённые количества посещений
df_data[df_data["visit_count"] == "не определено"]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_service_id,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,visit_count


In [118]:
# сохранение результатов на диск
df_data.to_csv("/home/sk27/work/med/data/result.csv", encoding="utf-8-sig")

## Исследование дубликатов
На примере денных Коми за февраль

In [41]:
with open("/home/sk27/work/med/new/11_komy/022023.csv", "r") as f:
    first_line = next(f).strip()
    if first_line[0:4] != "year":
        with open(
            "/home/sk27/work/med/new/11_komy/022023.csv", "r", encoding="ISO-8859-1"
        ) as original:
            data = original.read()
        with open("/home/sk27/work/med/new/11_komy/022023.csv", "w") as modified:
            modified.write(
                "year;month;region;clinic;oid;depart;branch;doctor_specialty;doctor_profile;terms;doctor_id;diagnosis;purpose;service_id;usl_service_id;usl_doctor_specialty;usl_clinic;usl_depart;sl_id;code_usl;kol_usl;yet_sum\n"
                + data
            )
    else:
        pass
df_test = pd.read_csv(
    "/home/sk27/work/med/new/11_komy/022023.csv",
    sep=";",
    encoding="ISO-8859-1",
    low_memory=False,
)

In [51]:
df_test = df_test.fillna(0)

In [52]:
df_test.shape

(146942, 22)

In [53]:
df_test[df_test.duplicated()].shape

(355, 22)

In [54]:
df_test_nodup = df_test.drop_duplicates()

In [55]:
df_test_nodup.shape

(146587, 22)

In [56]:
df_test_nodup

,year,month,region,clinic,oid,depart,branch,doctor_specialty,doctor_profile,terms,...,purpose,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum
0,2023,2,87000,110001,1.2.643.5.1.13.13.12.2.11.724,11000100,0.0,2,136,3,...,1.0,B01.001.001,035-508-952 59,2,110001,11000100,1,1,1,0.00
1,2023,2,87000,110001,1.2.643.5.1.13.13.12.2.11.724,11000100,0.0,2,136,3,...,1.0,B01.001.001,035-508-952 59,2,110001,11000100,4,1,1,0.00
2,2023,2,87000,110001,1.2.643.5.1.13.13.12.2.11.724,11000100,0.0,2,136,3,...,3.0,B01.001.001,035-508-952 59,2,110001,11000100,3,1,1,0.00
3,2023,2,87000,110001,1.2.643.5.1.13.13.12.2.11.724,11000100,0.0,2,136,3,...,3.0,B01.001.001,035-508-952 59,2,110001,11000100,2,2,1,0.00
4,2023,2,87000,110001,1.2.643.5.1.13.13.12.2.11.724,11000100,0.0,2,136,3,...,3.0,B01.001.001,073-069-750 73,2,110001,11000100,1,1,1,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146937,2023,2,87000,110065,1.2.643.5.1.13.13.12.2.11.778,11006500,0.0,208,85,3,...,3.0,A16.07.025.001,131-143-489 14,208,110065,11006500,1,1,1,0.20
146938,2023,2,87000,110065,1.2.643.5.1.13.13.12.2.11.778,11006500,0.0,208,85,3,...,3.0,A16.07.051,131-143-489 14,208,110065,11006500,1,1,1,2.00
146939,2023,2,87000,110065,1.2.643.5.1.13.13.12.2.11.778,11006500,0.0,208,85,3,...,3.0,B01.003.004.004,131-143-489 14,208,110065,11006500,1,1,1,0.31
146940,2023,2,87000,110065,1.2.643.5.1.13.13.12.2.11.778,11006500,0.0,208,85,3,...,3.0,B01.065.003,131-143-489 14,208,110065,11006500,1,1,1,1.68


In [49]:
df_test.columns

Index(['year', 'month', 'region', 'clinic', 'oid', 'depart', 'branch',
       'doctor_specialty', 'doctor_profile', 'terms', 'doctor_id', 'diagnosis',
       'purpose', 'service_id', 'usl_service_id', 'usl_doctor_specialty',
       'usl_clinic', 'usl_depart', 'sl_id', 'code_usl', 'kol_usl', 'yet_sum'],
      dtype='object')

In [57]:
df_test_group = df_test.groupby(
    [
        "year",
        "month",
        "region",
        "clinic",
        "oid",
        "depart",
        "branch",
        "doctor_specialty",
        "doctor_profile",
        "terms",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "usl_clinic",
        "usl_depart",
    ]
).agg({"sl_id": "sum", "code_usl": "sum", "kol_usl": "sum", "yet_sum": "sum"})

In [58]:
df_test_group = df_test_group.reset_index()

In [59]:
df_test.head(10)

,year,month,region,clinic,oid,depart,branch,doctor_specialty,doctor_profile,terms,...,purpose,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum
0,2023,2,87000,110001,1.2.643.5.1.13.13.12.2.11.724,11000100,0.0,2,136,3,...,1.0,B01.001.001,035-508-952 59,2,110001,11000100,1,1,1,0.0
1,2023,2,87000,110001,1.2.643.5.1.13.13.12.2.11.724,11000100,0.0,2,136,3,...,1.0,B01.001.001,035-508-952 59,2,110001,11000100,4,1,1,0.0
2,2023,2,87000,110001,1.2.643.5.1.13.13.12.2.11.724,11000100,0.0,2,136,3,...,3.0,B01.001.001,035-508-952 59,2,110001,11000100,3,1,1,0.0
3,2023,2,87000,110001,1.2.643.5.1.13.13.12.2.11.724,11000100,0.0,2,136,3,...,3.0,B01.001.001,035-508-952 59,2,110001,11000100,2,2,1,0.0
4,2023,2,87000,110001,1.2.643.5.1.13.13.12.2.11.724,11000100,0.0,2,136,3,...,3.0,B01.001.001,073-069-750 73,2,110001,11000100,1,1,1,0.0
5,2023,2,87000,110001,1.2.643.5.1.13.13.12.2.11.724,11000100,0.0,2,136,3,...,3.0,B01.001.002,035-508-952 59,2,110001,11000100,4,1,1,0.0
6,2023,2,87000,110001,1.2.643.5.1.13.13.12.2.11.724,11000100,0.0,2,136,3,...,3.0,B01.001.002,035-508-952 59,2,110001,11000100,1,4,1,0.0
7,2023,2,87000,110001,1.2.643.5.1.13.13.12.2.11.724,11000100,0.0,2,136,3,...,3.0,B01.001.001,035-508-952 59,2,110001,11000100,1,1,1,0.0
8,2023,2,87000,110001,1.2.643.5.1.13.13.12.2.11.724,11000100,0.0,2,136,3,...,3.0,B01.001.002,035-508-952 59,2,110001,11000100,1,2,1,0.0
9,2023,2,87000,110001,1.2.643.5.1.13.13.12.2.11.724,11000100,0.0,2,136,3,...,3.0,B01.001.001,035-508-952 59,2,110001,11000100,1,1,1,0.0


In [60]:
df_test_group.head(10)

,year,month,region,clinic,oid,depart,branch,doctor_specialty,doctor_profile,terms,...,purpose,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum
0,2022,9,87000,110035,1.2.643.5.1.13.13.12.2.11.758,11003505,0.0,79,100,3,...,3.0,B01.050.001,073-703-876 74,79,110035,11003505,1,1,1,0.0
1,2022,9,87000,110044,1.2.643.5.1.13.13.12.2.11.733,11004415,0.0,92,122,3,...,3.0,B01.058.001,056-635-703 76,92,110044,11004415,1,1,1,0.0
2,2022,11,87000,110044,1.2.643.5.1.13.13.12.2.11.733,11004415,0.0,92,122,3,...,3.0,B01.058.001,056-635-703 76,92,110044,11004415,1,1,1,0.0
3,2022,12,87000,110011,1.2.643.5.1.13.13.12.2.11.756,11001100,0.0,35,158,3,...,3.0,B01.031.002,005-922-854 44,104,110011,11001100,1,1,1,0.0
4,2022,12,87000,110016,1.2.643.5.1.13.13.12.2.11.702,11001600,0.0,79,100,3,...,3.0,B01.050.002,127-630-334 43,79,110016,11001600,1,2,1,0.0
5,2022,12,87000,110035,1.2.643.5.1.13.13.12.2.11.758,11003508,0.0,25,29,3,...,3.0,B01.015.001,127-325-262 41,25,110035,11003508,1,1,1,0.0
6,2022,12,87000,110037,1.2.643.5.1.13.13.12.2.11.738,11003708,0.0,17,16,3,...,3.0,B01.008.001,113-481-084 25,17,110037,11003708,1,1,1,0.0
7,2022,12,87000,110037,1.2.643.5.1.13.13.12.2.11.738,11003708,0.0,79,100,3,...,3.0,B01.050.001,152-504-576 43,79,110037,11003708,1,1,1,0.0
8,2022,12,87000,110049,1.2.643.5.1.13.13.12.2.11.752,11004944,0.0,76,97,3,...,3.0,B01.047.005,135-358-129 58,76,110049,11004944,1,1,1,0.0
9,2022,12,87000,110051,1.2.643.5.1.13.13.12.2.11.765,11005100,0.0,2,136,3,...,3.0,B01.001.004,215-943-213 57,2,110051,11005100,1,1,1,0.0


In [61]:
def visit_count(row):
    """
    функция определяет количество посещений согласно условиям
    """
    if row["purpose"] >= 3:
        res = row["code_usl"] * row["kol_usl"]
    else:
        res = row["sl_id"]
    if np.isnan(res):
        return "не определено"
    else:
        return res

In [62]:
df_test["visit_count"] = df_test.apply(visit_count, axis=1)

In [63]:
df_test_group["visit_count"] = df_test_group.apply(visit_count, axis=1)

In [64]:
df_test_nodup["visit_count"] = df_test.apply(visit_count, axis=1)

In [65]:
df_test["visit_count"].sum()

358095

In [66]:
df_test_group["visit_count"].sum()

483241

In [67]:
df_test_nodup["visit_count"].sum()

357656

In [68]:
df_test.shape

(146942, 23)

In [69]:
df_test_group.shape

(128501, 23)

In [70]:
df_test_nodup.shape

(146587, 23)

In [72]:
df_test.visit_count.describe()

count    146942.000000
mean          2.436982
std           8.576010
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max        1061.000000
Name: visit_count, dtype: float64

In [73]:
df_test_nodup.visit_count.describe()

count    146587.000000
mean          2.439889
std           8.586147
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max        1061.000000
Name: visit_count, dtype: float64

In [74]:
df_test_group.visit_count.describe()

count    128501.000000
mean          3.760601
std          13.629208
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max        1061.000000
Name: visit_count, dtype: float64

In [76]:
df_test.loc[1]

year                                             2023
month                                               2
region                                          87000
clinic                                         110001
oid                     1.2.643.5.1.13.13.12.2.11.724
depart                                       11000100
branch                                            0.0
doctor_specialty                                    2
doctor_profile                                    136
terms                                               3
doctor_id                              035-508-952 59
diagnosis                                       D25.9
purpose                                           1.0
service_id                                B01.001.001
usl_service_id                         035-508-952 59
usl_doctor_specialty                                2
usl_clinic                                     110001
usl_depart                                   11000100
sl_id                       

***

[Работа с Excel](https://realpython.com/openpyxl-excel-spreadsheets-python/)

In [4]:
from openpyxl import load_workbook

In [5]:
workbook = load_workbook(filename="/home/sk27/work/med/data/F032.xlsx")
workbook.sheetnames

['Справочник']

In [7]:
sheet = workbook.active
sheet

<Worksheet "Справочник">

In [14]:
for value in sheet.iter_rows(
    min_row=1, max_row=2, min_col=1, max_col=10, values_only=True
):
    print(value)

('UIDMO', 'IDMO', 'MCOD', 'OKTMO_P', 'SUBJ', 'D_BEGIN', 'D_END', 'NAME_E', 'DATEBEG', 'DATEEND')
('01202201101000000', '00003503000000007', '010078', '79000000000', 'Республика Адыгея', datetime.datetime(2021, 8, 19, 0, 0), None, None, datetime.datetime(2022, 1, 1, 0, 0), datetime.datetime(2022, 10, 11, 0, 0))
